# PART 04. 작물 잎 사진으로 질병 분류하기

## 데이터 분할

* 데이터 분할을 위한 디렉토리 생성

In [2]:
import os
import shutil
 
original_dataset_dir = './dataset'   # 데이터 셋 위치 경로
classes_list = os.listdir(original_dataset_dir) # 경로내 하위 폴더 목록을 가져와 클래스 목록으로 저장
 
base_dir = './splitted'  # 분류한 데이터 저장할 폴더 생성
os.mkdir(base_dir)
 
train_dir = os.path.join(base_dir, 'train') # 분리후 train, val,test 데이터 저장할 폴더 생성
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)#############################################################

for cls in classes_list:     # train, val, test 폴더 각각에 클래스 목록 폴더 생성
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: './splitted'

* 데이터 분할과 클래스별 데이터 수 확인

In [ ]:
import math
 
for cls in classes_list: # 모든 클래스에 대해 하위 작업들 반복
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path) # path 위치 내 모든 이미지 파일을 변수 fnames에 저장
 
    train_size = math.floor(len(fnames) * 0.6) # train, val, test 데이터 비율을 6:2:2 로 지정
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size] # train 데이터에 해당되는 파일이름을 train_fnames에 저장
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames: # 모든 train 데이터에 대해 하위 작업들 반복
        src = os.path.join(path, fname) # 원본 파일 경로
        dst = os.path.join(os.path.join(train_dir, cls), fname) # 복사본 저장 경로
        shutil.copyfile(src, dst) # 저장 실행
        
    validation_fnames = fnames[train_size:(validation_size + train_size)] # train 데이터에서 했던 작업과 동일
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)] # test 데이터에서 했던 작업과 동일

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)


## 베이스라인 모델 학습

* 베이스라인 모델 학습을 위한 준비

In [ ]:
import torch
import os
 
USE_CUDA = torch.cuda.is_available() # 현재 환경에서 GPU 사용 가능 여부 (True/False)
DEVICE = torch.device("cuda" if USE_CUDA else "cpu") # 현재 사용하는 장비를 저장(위에 코드가 True--> cuda/ False-->CPU)
BATCH_SIZE = 256 # 배치 사이즈 설정
EPOCH = 30 # 에폭 횟수 설정

In [2]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
 
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()]) 
# 이미지를 64*64로 조정하고, ToTensor를 사용해 이미지를 Tensor로 젼환해 값을 0~1 로 정규화 시키기위해 transforms.Compose 함수를 불러와 transform_base 변수 선언
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base) 
# ImageFolder 를 통해 데이터셋을 불러옵니다. root는 데이터를 불러올 경로이고 transform은 데이터를 불러온후 전처리 방법을 지정합니다.
#위에 정의한 transform_base를 사용합니다
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)
# 위와 같음

C:\Users\HP\miniconda3\envs\yoona\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [1]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# 불러온 데이터를 배치 단위로 불러옵니다. shuffle은 데이터의 순서를 섞어 가져오는지의 여부이고, num_workers는 서브프로세스의 개수입니다.
# num_workers default는 0 으로 이경우 main process에 모두 배정됩니다.
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# 위와 동일

NameError: name 'torch' is not defined

* 베이스라인 모델 설계하기

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module):  ## nn.Module 클래스를 상속하여 딥러닝 모델과 관련된 기본적인 함수를 사용이 가능해집니다.
  
    def __init__(self): # 모델에서 사용될 전체 layer 정의
    
        super(Net, self).__init__()  ## nn.Module 내 메서드 상속

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # 입력 채널:3 출력채널:32 커널크기:3, 패딩 1의 CONV2D layer
        self.pool = nn.MaxPool2d(2,2)   # 커널 사이즈2, stride 2의 maxplooing layer
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 입력 채널:32 출력채널:64 커널크기:3, 패딩 1의 CONV2D layer
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)   # 입력 채널:64 출력채널:64 커널크기:3, 패딩 1의 CONV2D layer

        self.fc1 = nn.Linear(4096, 512) # Flatten 이후 사용될 1번째 fully connected layer(Dense Layer)
        self.fc2 = nn.Linear(512, 33) v# Flatten 이후 사용될 2번쨰 fully connected layer(Dense Layer)
    
    def forward(self, x):  # 순전파 과정 정의
    
        x = self.conv1(x) # 1번쨰 Conv2d layer를 이용해 Feature Map 생성
        x = F.relu(x)  # 위에 Feature Map 에 relu 함수 적용
        x = self.pool(x) # 위에서 정의한 Maxpooling 적용
        x = F.dropout(x, p=0.25, training=self.training) # maxpooling 이후 결과값에 대한 연산. 노드의 25%를 Dropout 함
        # training은 학습 모드일때와 검증 모드일 때의 차이를 두기 위해 존재. 학습과정에선 DropOut 수행. 평가과정에선 x

        x = self.conv2(x) # 위와 같은 방식
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)  # 위와 같은 방식
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1, 4096)  # 생성된 Feature Map 1차원으로 펼치는 flatten 과정
        x = self.fc1(x)  # 위의 Output 을 1번째 fully connected layer(Dense Layer)에 통과 시킴
        x = F.relu(x)  # reLu 적용
        x = F.dropout(x, p=0.5, training=self.training)# dropout
        x = self.fc2(x)  # 위의 Output 을 2번째 fully connected layer(Dense Layer)에 통과 시킴. 출력 결과 33개

        return F.log_softmax(x, dim=1)  # 33개의 결괏값에 대해 softmax 함수를 적용하여 데이터가 각 클래스에 속할 확률을 출력

model_base = Net().to(DEVICE)   # 모델 객체 생성 후, 현재 사용 중인 장비에 할당
optimizer = optim.Adam(model_base.parameters(), lr=0.001)  # optimizer는 adam, learning rate는 0.001로 설정

* 모델 학습을 위한 함수

In [ ]:
def train(model, train_loader, optimizer):
    model.train()  # 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):# for 문으로 batch_idx, (data, target) 를 꺼내옴
        data, target = data.to(DEVICE), target.to(DEVICE) # data와 target 변수를 사용중인 장비에 할당
        optimizer.zero_grad() # optimizer 초기화(이전 Batch의 Gradient 값 초기화)
        output = model(data)   # output 값 계산
        loss = F.cross_entropy(output, target)# output값(모델에서 출력한 값) 과 target(실제값) 사이의 오차를 cross entropy loss를 사용하여 계산. 
        loss.backward()   # 위에서 계산한 loss값을 이용하여 역전파 실행. 계산된 Gradient 값을 각 parameter에 할당
        optimizer.step()  # 할당된 값을 통해 모델의 parameter 업데이트

* 모델 평가를 위한 함수

In [ ]:
def evaluate(model, test_loader):
    model.eval()  # 모델 평가 모드로 실행
    test_loss = 0 # 각 배치의 loss 값을 합산한 값 저장 변수
    correct = 0   # 올바르게 예측한 데이터 수  세는 변수
    
    with torch.no_grad(): # 모델 평가하는 동안엔 parameter 업데이트를 중단해줌
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE)  # 위와 동일
            output = model(data) #위와 동일
            
            test_loss += F.cross_entropy(output,target, reduction='sum').item() #위와 동일 
 
            
            pred = output.max(1, keepdim=True)[1] # 33개의 클래스에 속할 각각의 확률들 중 가장 큰 값의 인덱스를 예측값으로 저장
            correct += pred.eq(target.view_as(pred)).sum().item() #target tensor의 구조를 pred tensor와 같은 구조로 정렬
   
    test_loss /= len(test_loader.dataset) # loss의 평균값 구함
    test_accuracy = 100. * correct / len(test_loader.dataset)  # 정확도의 평균값 구함
    return test_loss, test_accuracy  # 측정한 test loss 와 정확도 반환

* 모델 학습을 실행하기

In [ ]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  # 가장 높은 정확도를 가진 모델의 정확도 저장
    best_model_wts = copy.deepcopy(model.state_dict())  # 가장 높은 정확도를 가진 모델 저장
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  # epoch 당 소요된 시간 측정을 위해 해당 epoch 시작 시간 저장
        train(model, train_loader, optimizer) # 모델 학습
        train_loss, train_acc = evaluate(model, train_loader) # 해당 epoch에서의 loss 값, 정확도 값 계산
        val_loss, val_acc = evaluate(model, val_loader) ## 해당 epoch에서의 loss 값, 정확도 값 계산
        
        if val_acc > best_acc:  # 현재 검증 정확도가 지금까지의 최고 정확도 보다 높다면
            best_acc = val_acc # 최고 정확도 값을 업데이트 하고
            best_model_wts = copy.deepcopy(model.state_dict()) # 해당 epoch의 모델을 best_model_wts에 저장
        
        time_elapsed = time.time() - since  # 하나의 epoch 당 소요 시간 계산. 시작 시간-epoch 끝난 시간
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))  # train loss, 정확도 출력  
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc)) # val loss, 정확도 출력
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # 에폭소요 시간 출력
    
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)  train_baseline() 함수 이용하여 baseline 모델 학습시킴
torch.save(base,'baseline.pt') # 학습 완료된 모델 저장

## Transfer Learning 모델 학습

* Transfer Learning을 위한 준비

In [ ]:
data_transforms = {
    'train': transforms.Compose([ #  train 데이터 전처리& augmentation.
        transforms.Resize([64,64]), # 이미지 크기 64 * 64 로 조정
        transforms.RandomHorizontalFlip(), # 이미지를 무작위로 좌우 반전
        transforms.RandomVerticalFlip(),   # 이미지를 무작위로 상하 반전
        transforms.RandomCrop(52),  # 이미지 일부를 랜덤하게 잘라내어 52 * 52 사이즈로 변경
        transforms.ToTensor(), # 이미지를 tensor로 변환, 0,1 사이 값으로 변경
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]), # 정규화 실행: 1번째 대괄호: rgb 채널에 적용할 평균값
        # 2번째 대괄호: rgb 채널에 적용될 표준편차 값
    
    'val': transforms.Compose([transforms.Resize([64,64]),  #  val 데이터 전처리& augmentation. 위와 이하 동문.
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [ ]:
data_dir = './splitted' # 학습 데이터와 검증 데이터를 불러올 경로 설정
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val']} 
# ImageFolder 를 통해 데이터셋을 불러옵니다. root는 데이터를 불러올 경로이고 transform은 데이터를 불러온후 전처리 방법을 지정합니다.
#앞서 정의한 transform_base를 사용합니다
# 딕셔너리 형태로 구성하여 불러옵니다
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']} 
# 불러온 데이터를 배치 단위로 불러옵니다. shuffle은 데이터의 순서를 섞어 가져오는지의 여부이고, num_workers는 서브프로세스의 개수입니다.
# num_workers default는 0 으로 이경우 main process에 모두 배정됩니다
# 딕셔너리 형태로 구성해 불러옵니다

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# 이후 활용을 위해 학습데이터와 검증데이터의 총 개수를 저장합니다.
# 딕셔너리 형태로 구성합니다.

class_names = image_datasets['train'].classes
# 33개의 클래스 목록의 이름을 지정합니다.

* Pre-Trained Model 불러오기

In [ ]:
from torchvision import models
 
resnet = models.resnet50(pretrained=True)  # ResNet 이미지 분류 모델을 parameter 값까지 가져 옵니다
num_ftrs = resnet.fc.in_features   # 모델에 채널 수가 33개인 layer를 새로 추가해 주기 위해 
#모델의 마지막 fully connected layer의 입력채널 수를 가져옵니다.
#in_featured는 해당 layer의 입력 채널수를 불러오는 함수입니다.

resnet.fc = nn.Linear(num_ftrs, 33) 
# 불러온 모델의 마지막 Fullt Connedted Layer의 입력채널수는 동일하지만 출력수는 33개인 layer를 새로 추가해줍니다.
resnet = resnet.to(DEVICE)
# 현재 사용중인 장비에 모델을 할당합니다.
 
criterion = nn.CrossEntropyLoss() # loss 함수를 cross entropy loss 사용
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)
# optimizer 는 adam, learning rate는 0.01로 설정 
# 일부 layer의 parameter만 업데이트 하기 위해 filter 메서드와 lamda 메서드를 사용하여
# requires_grad= true로 설정한 layer만 parameter 업데이트
 
from torch.optim import lr_scheduler 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) 
#StepLR()메서드는 Epoch에 따라 learning rate를 변경하는 역할을 합니다.
# 이 경우 7 epoch 마다 0.1 씩 곱해 learning rate를 감소시킨다는 의미 입니다.

* Pre-Trained Model의 일부 Layer Freeze하기

In [ ]:
ct = 0 # 해당layer가 몇번째 layer인지 나타내는 변수 ct의 값을 0으로 초기화
for child in resnet.children():  # children() 메서드는 모델의 자식 모튤을 반복 가능한 객체로 반환하는 메서드. resnet모델의 모든 layer정보를 담고 있음
    ct += 1  # for문 1번 반복후 다음 layer를 가리킵니다.
    if ct < 6: # 1~5번 layer는
        for param in child.parameters(): # chilf.parameters()는 각 layer의 parameter tensor를 의미합니다. 각 tensor에는 requires_grad 옵션이 있고, 기본값은 true 로 설정되어 있습니다.
            param.requires_grad = False # parameter를 업데이트 하지 않습니다

* Transfer Learning 모델 학습과 검증을 위한 함수

In [ ]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())  # 정확도가 가장 높은 모델을 저장할 변수 best_model_wts를 선언
    best_acc = 0.0  # 정확도가 가장 높은 모델의 정확도를 저장하는 변수 best_acc를 선언하고, 값을 0으로 초기화
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1)) # 현재 진행중인 에폭 출력
        since = time.time() # 한 에폭 당 소요되는 시간을 측정하기 위해 해당 에폭을 시작할 떄의 시각을 저장합니다.                                    
        for phase in ['train', 'val']: # 한 에폭마다 학습모드와 검증 모드를 각각 실행합니다
            if phase == 'train': # 상황에 적합하게 모델을 학습 모드로 설정합니다
                model.train() 
            else:
                model.eval()     # 상황에 적합하게 모델을 검증 모드로 설정합니다.
 
            running_loss = 0.0  # 모든 데이터의 loss를 합산해 저장할 변수 running_loss 선언 후 0.0으로 초기화
            running_corrects = 0  # 올바르게 예측한 경우의 수를 세는 변수인 running_corrects를 선언, 0으로 초기화
 
            
            for inputs, labels in dataloaders[phase]: # 모델의 현재 모드에 해당하는 Dataloader에서 데이터를 입력받습니다.
                inputs = inputs.to(DEVICE)  # 데이터와 해당하는 label 값을 현재 사용중인 장비에 할당합니다.
                labels = labels.to(DEVICE)  # 데이터와 해당하는 label 값을 현재 사용중인 장비에 할당합니다.
                
                optimizer.zero_grad() # optimizer 초기화(이전 Batch의 Gradient 값 초기화)
                
                with torch.set_grad_enabled(phase == 'train'): # 학습단계에서만 모델의 gradient를 업데이트 하도록 set_grad_enabled 메서드 사용 
                    outputs = model(inputs)  # 데이터를 모델에 입력하여 output 값 계산
                    _, preds = torch.max(outputs, 1) # 33개의 클래스에 속할 각각의 확률들 중 가장 큰 값의 인덱스를 예측값으로 저장
                    loss = criterion(outputs, labels)  # output값과 target 값 사이이 loss를 계산합니다
    
                    if phase == 'train':   # 모델이 학습 모드인 경우에만
                        loss.backward() # 역전파를 실행하고
                        optimizer.step() # 모델의 parameter 값을 업데이트 합니다.
 
                running_loss += loss.item() * inputs.size(0)  # 모든 데이터 loss를 합산해 저장합니다. inputs.size(0)는 dataloader에 전달되는 미니배치의 데이터수를 의미합니다
                running_corrects += torch.sum(preds == labels.data)  # 모델을 동해 예측한 값과 target이 같으면 running_corrects를 1만큼 증가시키고, 같지 않으면 증가시키지 않습니다.
            if phase == 'train': # 7 에폭마다 learning rate를 다르게 조정하는 scheduler와 관련된 부분입니다. 한 epoch 마다 1번, 모델이 현재 학습 단계일 경우에만 실행 
                scheduler.step()
                
                l_r =[x['lr'] for x in optimizerz-ft.param_groups] #scheduler에 의해 learning rate가 조정되는 것을 직접확인
                # optimizer_ft.param_groups : 학습 과정에서는 parameter를 저장하고 있는 딕셔너리
                # 키값 lr 을 이용하여 각 epoch의 learning rate를 불러옴
                print('learning rate: ', l_r) 
 
            epoch_loss = running_loss/dataset_sizes[phase]   # 해당 epoch의 loss 계산
            epoch_acc = running_corrects.double()/dataset_sizes[phase]  # 해당 epoch의 정확도 계산
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))  # 해당 eopch과 현재 모델의 단계, 정확도 출력
 
          
            if phase == 'val' and epoch_acc > best_acc: # 검증 단계에서 현재 epoch의 정확도가 최고 정확도 보다 높다면
                best_acc = epoch_acc # 최고 정확도 업데이트  하고
                best_model_wts = copy.deepcopy(model.state_dict()) # 해당 epoch의 모델을 best_model_wts에 저장
 
        time_elapsed = time.time() - since  # 한 epoch 당 소요된 시간 계산
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # 소요시간 출력
    print('Best val Acc: {:4f}'.format(best_acc)) #  최고 정확도 출력
 
    model.load_state_dict(best_model_wts) # 정확도가 가장 높은 모델을 불러온 후

    return model #반환

* 모델 학습을 실행하기

In [ ]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH) 
# 앞서 정의한 train_resnet 함수를 이용하여 RestNet50 모델을 Fine-Tuning 합니다.
torch.save(model_resnet50, 'resnet50.pt')
# 학습이 완료된 모델을 'resnet50.pt란 이름으로 저장합니다'

## 모델 평가

* 베이스라인 모델 평가를 위한 전처리하기

In [ ]:
transform_base = transforms.Compose([transforms.Resize([64,64]),transforms.ToTensor()])
# 이미지를 64*64로 조정하고, ToTensor를 사용해 이미지를 Tensor로 젼환해 값을 0~1 로 정규화 시키기위해 transforms.Compose 함수를 불러와 transform_base 변수 선언
test_base = ImageFolder(root='./splitted/test',transform=transform_base)  
# ImageFolder 를 통해 데이터셋을 불러옵니다. root는 데이터를 불러올 경로이고 transform은 데이터를 불러온후 전처리 방법을 지정합니다.
#위에 정의한 transform_base를 사용합니다
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# 불러온 데이터를 배치 단위로 불러옵니다. shuffle은 데이터의 순서를 섞어 가져오는지의 여부이고, num_workers는 서브프로세스의 개수입니다.
# num_workers default는 0 으로 이경우 main process에 모두 배정됩니다.

* Transfer Learning모델 평가를 위한 전처리하기

In [ ]:
transform_resNet = transforms.Compose([ #train 데이터 전처리& augmentation.
        transforms.Resize([64,64]),   # 이미지 크기 64 * 64 로 조정
        transforms.RandomCrop(52),    # 이미지 일부를 랜덤하게 잘라내어 52 * 52 사이즈로 변경
        transforms.ToTensor(),        # 이미지를 tensor로 변환, 0,1 사이 값으로 변경
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화 실행: 1번째 대괄호: rgb 채널에 적용할 평균값
        # 2번째 대괄호: rgb 채널에 적용될 표준편차 값
    ])
    
test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet) 
# ImageFolder 를 통해 데이터셋을 불러옵니다. root는 데이터를 불러올 경로이고 transform은 데이터를 불러온후 전처리 방법을 지정합니다.
#위에 정의한 transform_resNet을 사용합니다
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
# 불러온 데이터를 배치 단위로 불러옵니다. shuffle은 데이터의 순서를 섞어 가져오는지의 여부이고, num_workers는 서브프로세스의 개수입니다.
# num_workers default는 0 으로 이경우 main process에 모두 배정됩니다.

* 베이스라인 모델 성능 평가하기

In [ ]:
baseline=torch.load('baseline.pt') # 저장된 베이스라인 모델 불러옴
baseline.eval()   # 모델을 평가 모드로 실행
test_loss, test_accuracy = evaluate(baseline, test_loader_base)
 # 앞서 정의한 evaluate 함수를 이용하요 테스트 데이터에 대한 정확도 측정
print('baseline test acc:  ', test_accuracy)# 평가 정확도 출력

* Transfer Learning 모델 성능 평가하기

In [ ]:
resnet50=torch.load('resnet50.pt') # 저장된 transfer learning 모델 불러옴
resnet50.eval()   # 모델을 평가 모드로 실행
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet) # 앞서 정의한 evaluate 함수를 이용하요 테스트 데이터에 대한 정확도 측정

print('ResNet test acc:  ', test_accuracy) # 평가 정확도 출력